# Дипломная работа по NLP

«Создание аналитической информационной системы для выделения ключевых слов из учебных материалов»

## Выполнил: студент группы ПИ20-5, Валяев Георгий Анатольевич

В этом скрипте будут реализованы процессы:
- количественное распределение РПД по годам и направлениям
- анализ объёма текстовой информации РПД после парсинга её ключевых разделов
- анализ популярности тем среди РПД
- реализация метрики относительной близости двух РПД с демонстрацией работы (выявление взаимосвязей между темами + анализ изменений в содержании дисциплин)
- дополнительные визуализации результатов

**Все изменения и версии ноутбука хранятся на репозитории GitHub:
https://github.com/Valyaevgeorgiy/VKR_work**

In [1]:
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 3.1 MB/s eta 0:00:00


In [341]:
import os, json, pprint, random, itertools
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from collections import Counter
from itertools import chain
from google.colab import drive

# модуль для работы с вордовскими документами и их содержимым (тексты, таблицы)
import docx

In [36]:
PATH = '/content/gdrive/MyDrive/Диплом_Валяев/dataset'

docs_2020_path = PATH+'/2020_year/'
docs_2021_path = PATH+'/2021_year/'
docs_2022_path = PATH+'/2022_year/'
docs_2023_path = PATH+'/2023_year/'
docs_engl = PATH+'/Англ РПД/'
docs_others = PATH+'/ГИА, практика и т.п./'

PATH_data = '/content/gdrive/MyDrive/Диплом_Валяев/data'
docs_rpd_data = PATH_data+'/rpd_data.json'
docs_rpd_keywords = PATH_data+'/rpd_keywords.json'

links_docs = '/content/gdrive/MyDrive/Диплом_Валяев/data/gd-discips-links.json'

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


## Количественная аналитика

In [37]:
files_2020 = os.listdir(docs_2020_path)
files_2021 = os.listdir(docs_2021_path)
files_2022 = os.listdir(docs_2022_path)
files_2023 = os.listdir(docs_2023_path)
files_others = os.listdir(docs_others)
files_engl = os.listdir(docs_engl)

### Распределение РПД по годам

In [5]:
rpd_data = [files_2020, files_2021, files_2022, files_2023, files_others, files_engl]
print(f"2020 год — {len(rpd_data[0])}")
print(f"2021 год — {len(rpd_data[1])}")
print(f"2022 год — {len(rpd_data[2])}")
print(f"2023 год — {len(rpd_data[3])}")
print()
print(f"Итоговое количество РПД (после очистки .doc) в рамках датасета = {len(rpd_data[0]) + len(rpd_data[1]) + len(rpd_data[2]) + len(rpd_data[3])}")
print(f"Количество документов по практике, ГИА и т.п. в рамках датасета = {len(rpd_data[4]) + len(rpd_data[5])}")
print()

fig = px.line(y=[len(rpd_data[0]), len(rpd_data[1]), len(rpd_data[2]), len(rpd_data[3]), len(rpd_data[4]), len(rpd_data[5])],
              x=['2020 год', '2021 год', '2022 год', '2023 год', 'ГИА, практика', 'Англ РПД'])
fig.update_layout(xaxis=dict(title='Группа РПД'), yaxis=dict(title='Количество документов'))
fig.show()

2020 год — 5
2021 год — 89
2022 год — 129
2023 год — 183

Итоговое количество РПД (после очистки .doc) в рамках датасета = 406
Количество документов по практике, ГИА и т.п. в рамках датасета = 76



### Распределение РПД по направлениям

In [6]:
def get_tituls_rpd(docs_path, files_path, by_year=False, by_names=False):
    """
    Парсинг титульных страниц РПД года на предмет
    выявления направления подготовки с дальнейшей агрегацией.
    «by_year» — условие парсинга в рамках одного года.
    «by_names» — условие вывода в итоговый словарь названий РПД с их количеством.
    return: dict
    """
    tituls_data = []
    docs = []
    repeat_docs_path = lambda doc_path, repeats=1: bool(result:=[]) or [result.extend(e) for e in zip(*([doc_path] * repeats))] and result

    if by_year:
      docs = [[docum, docx.Document(docs_path + docum)] for docum in files_path]
    else:
      docs_path = [repeat_docs_path([docs_path[ind]], len(files_path[ind])) for ind in range(len(docs_path))]
      for docs_path_y, files_path_y in zip(docs_path, files_path):
        docs += [[docum, docx.Document(docum_path_y + docum)] for docum_path_y, docum in zip(docs_path_y, files_path_y)]

    for (ind, (name, doc)) in enumerate(docs):
      titul = [p.text.lower().strip().replace('-', ' ').replace('\n', ' ').replace('\t', ' ') for p in doc.paragraphs[6:30] if p.text != '' and p.text != ' ']
      titul = [data for data in titul if ((len(data) > 0 and data[0].isdigit()) or (data.startswith('по направлению')) or (data.startswith('для студентов, обучающихся по направлению подготовки ')) or (data.startswith('дисциплина «технологии математического моделирования и анализа данных')))]
      titul = ' '.join(titul).strip().replace('«', '').replace('»', '').replace(',', '').replace('–', '').replace('\t', '').split()
      end_ind = len(titul)

      for ind, data in enumerate(titul):
        if data.count('.', 2) and len(data) == 8:
          key_ind = ind
        elif data == 'оп' or data.startswith('направленность') or data.startswith('1.'):
          end_ind = ind
      titul = [titul[key_ind:end_ind][0], ' '.join(titul[key_ind:end_ind][1:])]
      tituls_data.append({name: titul})

    count_napravs = {napr: [0, []] for napr in set([list(titul.values())[0][1] for titul in tituls_data])}
    for titul in tituls_data:
      count_napravs[list(titul.values())[0][1]][0] += 1
      count_napravs[list(titul.values())[0][1]][1].append(list(titul.keys())[0])
    count_napravs = {napr: [count, sorted(f)] if by_names else count for napr, (count, f) in count_napravs.items()}
    return count_napravs

#### Агрегация по конкретному году

##### 2020 год

In [7]:
rpd_naprs_2020 = get_tituls_rpd(docs_2020_path, files_2020, by_year=True, by_names=True)
pprint.pprint(rpd_naprs_2020)

{'экономика': [5,
               ['Методы ВД,Эк,КФУ.docx',
                'Нейронные сети,Эк, КФУ.docx',
                'Обработка данных и мод. в МЕ,Эк, КФУ.docx',
                'Прикл задачи МОиОБД, Эк, КФУ.docx',
                'Програм. на языках PYTHON и SQL,Эк, КФУ.docx']]}


In [8]:
print(f'Направление подготовки — «{list(rpd_naprs_2020.keys())[0].capitalize()}»\nКоличество РПД — {list(rpd_naprs_2020.values())[0][0]} документов')

Направление подготовки — «Экономика»
Количество РПД — 5 документов


##### 2021 год

In [9]:
rpd_naprs_2021 = get_tituls_rpd(docs_2021_path, files_2021, by_year=True, by_names=True)
pprint.pprint(rpd_naprs_2021)

{'бизнес информатика': [4,
                        ['Алгоритмы и СД в яз Python, БИ.docx',
                         'Базы данных, БИ.docx',
                         'Математика, БИ.docx',
                         'ТВиМС, БИ.docx']],
 'инноватика': [2,
                ['Базы данных, Ин.docx', 'Прогр и АД с пом PYTHON, Ин.docx']],
 'информатика и вычислительная техника': [5,
                                          ['Агент мод.docx',
                                           'Сложные СиГ (ИВТ, 2021).docx',
                                           'Соврем ТИАТиДС, ИВТ, 2021.docx',
                                           'Соврем тех ОБД,.docx',
                                           'Современные ТКЗ, ИВТ.docx']],
 'информационная безопасность': [6,
                                 ['Алгоритмы и СД в яз Python, ИБ.docx',
                                  'Диск мат, ИБ.docx',
                                  'Математика, ИБ.docx',
                                  'ТВиМС,ИБ.docx'

In [21]:
data = sorted([(napr, val[0]) for napr, val in rpd_naprs_2021.items()], key=lambda x: x[0])

# Разделение списка на отдельные категории и их количества
categories = [item[0] for item in data]
counts = [item[1] for item in data]

fig = px.bar(pd.DataFrame(data, columns=['Направления', 'Число РПД']), x='Направления', y='Число РПД', color=categories)
fig.update_layout(title=dict(text='РПД 2021 года', x=0.5))
fig.show()

##### 2022 год

In [11]:
rpd_naprs_2022 = get_tituls_rpd(docs_2022_path, files_2022, by_year=True, by_names=True)
pprint.pprint(rpd_naprs_2022)

{'бизнес информатика': [4,
                        ['Алг и СД в яз Python (БИ, 2022).docx',
                         'Мат методы ПР, БИ, 2022.docx',
                         'Математика (БИ, 2022).docx',
                         'ТВиМС, БИ, 2022.docx']],
 'инноватика': [3,
                ['Базы данных (Ин, 2022).docx',
                 'Машинное обуч и ИИвБ, 2021, 2022.docx',
                 'П и АД с пом Python, Ин, 2022.docx']],
 'информатика и вычислительная техника': [2,
                                          ['ММЧМиКП, 2022.docx',
                                           'Современные ТГО, ИВТ.docx']],
 'информационная безопасность': [6,
                                 ['Алг и СД в яз Python (ИБ, 2022).docx',
                                  'Дискрет матем (ИБ, 2022).docx',
                                  'Математика (ИБ, 2022).docx',
                                  'ТВиМС, ИБ, 2022.docx',
                                  'ФУ_ИиС (ИБ,2022).docx',
                     

In [20]:
data = sorted([(napr, val[0]) for napr, val in rpd_naprs_2022.items()], key=lambda x: x[0])

# Разделение списка на отдельные категории и их количества
categories = [item[0] for item in data]
counts = [item[1] for item in data]

fig = px.bar(pd.DataFrame(data, columns=['Направления', 'Число РПД']), x='Направления', y='Число РПД', color=categories)
fig.update_layout(title=dict(text='РПД 2022 года', x=0.5))
fig.show()

##### 2023 год

In [13]:
rpd_naprs_2023 = get_tituls_rpd(docs_2023_path, files_2023, by_year=True, by_names=True)
pprint.pprint(rpd_naprs_2023)

{'бизнес информатика': [2,
                        ['А и СД в яз Python, БИ, 2023.docx',
                         'Основы ТИВ, БИ, 2023.docx']],
 'инноватика': [4,
                ['Базы данных, 2023.docx',
                 'Введение в НоД, Ин, 2023.docx',
                 'Основы МиР, Ин,  2021, 2022.docx',
                 'Проектирование СЭК, 2021, 2022.docx']],
 'информационная безопасность': [8,
                                 ['А и СД в яз Python, ИБ, 2023.docx',
                                  'Автомат БС, 2021, 2022.docx',
                                  'Автомат системы БУАиАвКО, 2021, 2022.docx',
                                  'Компьютерное МЭиФС, ИБ, 2023.docx',
                                  'Математика, ИБ, 2023.docx',
                                  'Прикладные ЗМО, ИБ, 2023.docx',
                                  'Фин универ, ИБ, 2023.docx',
                                  'Финтех, ИБ, 2023.docx']],
 'лингвистика': [2,
                 ['Интерактивные ТСК

In [19]:
data = sorted([(napr, val[0]) for napr, val in rpd_naprs_2023.items()], key=lambda x: x[0])

# Разделение списка на отдельные категории и их количества
categories = [item[0] for item in data]
counts = [item[1] for item in data]

fig = px.bar(pd.DataFrame(data, columns=['Направления', 'Число РПД']), x='Направления', y='Число РПД', color=categories)
fig.update_layout(title=dict(text='РПД 2023 года', x=0.5))
fig.show()

#### Агрегация по всем годам

In [15]:
rpd_naprs_all = get_tituls_rpd([docs_2020_path, docs_2021_path, docs_2022_path, docs_2023_path], [files_2020, files_2021, files_2022, files_2023], by_names=True)
pprint.pprint(rpd_naprs_all)

{'бизнес информатика': [10,
                        ['А и СД в яз Python, БИ, 2023.docx',
                         'Алг и СД в яз Python (БИ, 2022).docx',
                         'Алгоритмы и СД в яз Python, БИ.docx',
                         'Базы данных, БИ.docx',
                         'Мат методы ПР, БИ, 2022.docx',
                         'Математика (БИ, 2022).docx',
                         'Математика, БИ.docx',
                         'Основы ТИВ, БИ, 2023.docx',
                         'ТВиМС, БИ, 2022.docx',
                         'ТВиМС, БИ.docx']],
 'инноватика': [9,
                ['Базы данных (Ин, 2022).docx',
                 'Базы данных, 2023.docx',
                 'Базы данных, Ин.docx',
                 'Введение в НоД, Ин, 2023.docx',
                 'Машинное обуч и ИИвБ, 2021, 2022.docx',
                 'Основы МиР, Ин,  2021, 2022.docx',
                 'П и АД с пом Python, Ин, 2022.docx',
                 'Прогр и АД с пом PYTHON, Ин.docx',
    

In [16]:
data = sorted([(napr, val[0]) for napr, val in rpd_naprs_all.items()], key=lambda x: x[0])

# Разделение списка на отдельные категории и их количества
categories = [item[0] for item in data]
counts = [item[1] for item in data]

fig = px.bar(pd.DataFrame(data, columns=['Направления', 'Число РПД']), x='Направления', y='Число РПД', color=categories)
fig.update_layout(title=dict(text='Распределение РПД за последние 4 года', x=0.5))
fig.show()

### Анализ объёма текстовой информации РПД

Важно понимать, что за расчёт берётся не весь файл РПД, а только его ключевые разделы:
- Содержание
- Вопросы к контрольной работе
- Вопросы для подготовки к зачёту/экзамену
- Вопросы для изучения на семинарских занятиях

In [22]:
with open(docs_rpd_data, 'r', encoding='utf-8') as file:
  rpd_data = json.load(file)

total_sent = []
total_words = []

for ind, disc in enumerate(list(rpd_data.keys())):
  num_sent = len(rpd_data[disc]['SODERZ'] + rpd_data[disc]['QUESKR'] + rpd_data[disc]['QUESSESS'] + rpd_data[disc]['QUESSEMS']) if 'QUESKR' in rpd_data[disc].keys() else len(rpd_data[disc]['SODERZ'] + rpd_data[disc]['QUESSESS'] + rpd_data[disc]['QUESSEMS'])
  num_words = sum([len(sent) for sent in list(rpd_data[disc]['SODERZ'] + rpd_data[disc]['QUESKR'] + rpd_data[disc]['QUESSESS'] + rpd_data[disc]['QUESSEMS'])]) if 'QUESKR' in rpd_data[disc].keys() else sum([len(sent) for sent in list(rpd_data[disc]['SODERZ'] + rpd_data[disc]['QUESSESS'] + rpd_data[disc]['QUESSEMS'])])

  total_sent.append(num_sent)
  total_words.append(num_words)

print(f'В сумме у датасета {sum(total_sent):,} предложений и {sum(total_words):,} слов.')
print(f'В среднем в 1 дисциплине содержится {int(round(sum(total_sent) / len(total_sent), 0)):,} предложений и {int(round(sum(total_words) / len(total_words), 0)):,} слов.')

В сумме у датасета 47,771 предложений и 4,491,636 слов.
В среднем в 1 дисциплине содержится 118 предложений и 11,063 слов.


## Популярность тем среди РПД

In [356]:
def get_popular_themes(kw_dataset):
  keywords_all_list = list(chain.from_iterable(kw_dataset))
  topic_counter = Counter(keywords_all_list)

  count_keywords = 30 if len(kw_dataset) >= 10 else 10
  counter_keywords = []

  print("Наиболее популярные темы/слова:")
  for (ind, (topic, count)) in enumerate(topic_counter.most_common(count_keywords)):
    counter_keywords.append((topic, count))
    print(f"{ind+1}. {topic.capitalize()} ({count} упоминаний(-я))")

  return counter_keywords

def vizual_popular_themes(counter_kw_data, name_hist):

  data = sorted(counter_kw_data, key=lambda x: -x[1])

  # Разделение списка на отдельные категории и их количества
  categories = [item[0] for item in data]
  counts = [item[1] for item in data]

  fig = px.bar(pd.DataFrame(data, columns=['Тема/Слово', 'Частота встречаемости']), x='Тема/Слово', y='Частота встречаемости')
  fig.update_layout(title=dict(text=name_hist, x=0.5))
  fig.show()

  return None

with open(docs_rpd_keywords, 'r', encoding='utf-8') as file:
  rpd_kw_data = json.load(file)

rpd_kw_data

{'Прикл задачи МОиОБД, Эк, КФУ.docx': ['spark',
  'hadoop',
  'кластер',
  'pandas',
  'yarn',
  'библиотека',
  'решающий',
  'numpy',
  'машинный',
  'анализ',
  'hdinsight',
  'rdd',
  'дерево',
  'интерактивный',
  'learn'],
 'Методы ВД,Эк,КФУ.docx': ['диаграмма',
  'гистограмма',
  'визуальный',
  'линейчатый',
  'microsoft',
  'power',
  'bi',
  'накопление',
  'визуализация',
  'office',
  'сводный',
  'excel',
  'группировка',
  'нормировать',
  'таблица'],
 'Програм. на языках PYTHON и SQL,Эк, КФУ.docx': ['sql',
  'python',
  'язык',
  'подзапрос',
  'использование',
  'библиотека',
  'sklearn',
  'таблица',
  'соединение',
  'процедура',
  'pandas',
  'numpy',
  'инструкция',
  'конструкция',
  'запрос'],
 'Нейронные сети,Эк, КФУ.docx': ['сеть',
  'нейронный',
  'learning',
  'нейрон',
  'архитектура',
  'networks',
  'neural',
  'тренировка',
  'искусственный',
  'pytorch',
  'радиальный',
  'персептрон',
  'смещение',
  'биологический',
  'базисный'],
 'Обработка данных и 

### Популярные темы по годам

#### 2020 год

In [357]:
counter_keywords_2020 = get_popular_themes([value for key, value in rpd_kw_data.items() if key in files_2020])
print()
vizual_popular_themes(counter_keywords_2020, 'Популярные темы/слова в РПД 2020 года')

Наиболее популярные темы/слова:
1. Таблица (3 упоминаний(-я))
2. Pandas (2 упоминаний(-я))
3. Библиотека (2 упоминаний(-я))
4. Numpy (2 упоминаний(-я))
5. Диаграмма (2 упоминаний(-я))
6. Сводный (2 упоминаний(-я))
7. Excel (2 упоминаний(-я))
8. Spark (1 упоминаний(-я))
9. Hadoop (1 упоминаний(-я))
10. Кластер (1 упоминаний(-я))



#### 2021 год

In [358]:
counter_keywords_2021 = get_popular_themes([value for key, value in rpd_kw_data.items() if key in files_2021])
print()
vizual_popular_themes(counter_keywords_2021, 'Популярные темы/слова в РПД 2021 года')

Наиболее популярные темы/слова:
1. Технология (12 упоминаний(-я))
2. Финансовый (11 упоминаний(-я))
3. Сеть (10 упоминаний(-я))
4. Линейный (9 упоминаний(-я))
5. Кластеризация (9 упоминаний(-я))
6. Язык (9 упоминаний(-я))
7. Машинный (9 упоминаний(-я))
8. Вектор (9 упоминаний(-я))
9. Проект (8 упоминаний(-я))
10. Python (8 упоминаний(-я))
11. Регрессия (8 упоминаний(-я))
12. Блокчейн (8 упоминаний(-я))
13. Формула (8 упоминаний(-я))
14. Цифровой (8 упоминаний(-я))
15. Информационный (7 упоминаний(-я))
16. Инструкция (7 упоминаний(-я))
17. Строка (7 упоминаний(-я))
18. Анализ (7 упоминаний(-я))
19. Криптовалюта (7 упоминаний(-я))
20. Финтех (7 упоминаний(-я))
21. Отрасль (7 упоминаний(-я))
22. Трансформация (7 упоминаний(-я))
23. Текст (7 упоминаний(-я))
24. Банк (7 упоминаний(-я))
25. Бизнес (7 упоминаний(-я))
26. Моделирование (6 упоминаний(-я))
27. Величина (6 упоминаний(-я))
28. Генератор (6 упоминаний(-я))
29. Нейронный (6 упоминаний(-я))
30. Уравнение (6 упоминаний(-я))



#### 2022 год

In [359]:
counter_keywords_2022 = get_popular_themes([value for key, value in rpd_kw_data.items() if key in files_2022])
print()
vizual_popular_themes(counter_keywords_2022, 'Популярные темы/слова в РПД 2022 года')

Наиболее популярные темы/слова:
1. Python (17 упоминаний(-я))
2. Сеть (15 упоминаний(-я))
3. Машинный (14 упоминаний(-я))
4. Анализ (13 упоминаний(-я))
5. Нейронный (13 упоминаний(-я))
6. Регрессия (11 упоминаний(-я))
7. Технология (11 упоминаний(-я))
8. Классификация (11 упоминаний(-я))
9. Язык (11 упоминаний(-я))
10. Библиотека (11 упоминаний(-я))
11. Цифровой (11 упоминаний(-я))
12. Архитектура (10 упоминаний(-я))
13. Изображение (10 упоминаний(-я))
14. Линейный (9 упоминаний(-я))
15. Модуль (9 упоминаний(-я))
16. Разработка (8 упоминаний(-я))
17. Уравнение (8 упоминаний(-я))
18. Моделирование (8 упоминаний(-я))
19. Проект (8 упоминаний(-я))
20. Принцип (8 упоминаний(-я))
21. Финансовый (8 упоминаний(-я))
22. Переменный (7 упоминаний(-я))
23. Порядок (7 упоминаний(-я))
24. Теорема (7 упоминаний(-я))
25. Алгоритм (7 упоминаний(-я))
26. Таблица (7 упоминаний(-я))
27. Использование (7 упоминаний(-я))
28. База (7 упоминаний(-я))
29. Формат (7 упоминаний(-я))
30. Метрика (7 упоминаний(-я

#### 2023 год

In [360]:
counter_keywords_2023 = get_popular_themes([value for key, value in rpd_kw_data.items() if key in files_2023])
print()
vizual_popular_themes(counter_keywords_2023, 'Популярные темы/слова в РПД 2023 года')

Наиболее популярные темы/слова:
1. Анализ (22 упоминаний(-я))
2. Машинный (20 упоминаний(-я))
3. Технология (18 упоминаний(-я))
4. Python (18 упоминаний(-я))
5. Информационный (17 упоминаний(-я))
6. Язык (17 упоминаний(-я))
7. Финансовый (17 упоминаний(-я))
8. Сеть (14 упоминаний(-я))
9. Граф (14 упоминаний(-я))
10. Веб (14 упоминаний(-я))
11. Приложение (14 упоминаний(-я))
12. Моделирование (14 упоминаний(-я))
13. Цифровой (13 упоминаний(-я))
14. Проектирование (13 упоминаний(-я))
15. Алгоритм (12 упоминаний(-я))
16. Тестирование (12 упоминаний(-я))
17. Диаграмма (12 упоминаний(-я))
18. Стиль (12 упоминаний(-я))
19. Модуль (12 упоминаний(-я))
20. Компьютерный (11 упоминаний(-я))
21. Библиотека (11 упоминаний(-я))
22. Создавать (11 упоминаний(-я))
23. Сервис (11 упоминаний(-я))
24. Разработка (10 упоминаний(-я))
25. Использование (10 упоминаний(-я))
26. Инструкция (10 упоминаний(-я))
27. Css (10 упоминаний(-я))
28. Предприятие (10 упоминаний(-я))
29. Процесс (10 упоминаний(-я))
30. Рег

### Популярные темы в направлениях

In [361]:
counter_keywords_all_naprs = []
for napr in rpd_naprs_all.keys():
  print(f'Направление — «{napr.capitalize()}»')
  count_keywords = 30 if rpd_naprs_all[napr][0] >= 10 else 10
  keywords_all_list = list(chain.from_iterable([value for key, value in rpd_kw_data.items() if key in rpd_naprs_all[napr][1]]))
  topic_counter = Counter(keywords_all_list)
  counter_napr = []

  print("Наиболее популярные темы/слова:")
  for (ind, (topic, count)) in enumerate(topic_counter.most_common(count_keywords)):
    counter_napr.append((topic, count))
    print(f"{ind+1}. {topic.capitalize()} ({count} упоминаний(-я))")

  counter_keywords_all_naprs.append((napr.capitalize(), counter_napr))
  print()
  print()

Направление — «Менеджмент»
Наиболее популярные темы/слова:
1. Технология (3 упоминаний(-я))
2. Проект (3 упоминаний(-я))
3. Сервис (3 упоминаний(-я))
4. Логистика (3 упоминаний(-я))
5. Запас (3 упоминаний(-я))
6. Диаграмма (3 упоминаний(-я))
7. Анализ (3 упоминаний(-я))
8. Финтех (2 упоминаний(-я))
9. Финансовый (2 упоминаний(-я))
10. Цифровой (2 упоминаний(-я))
11. Бизнес (2 упоминаний(-я))
12. Отрасль (2 упоминаний(-я))
13. Блокчейн (2 упоминаний(-я))
14. Биткойный (2 упоминаний(-я))
15. Криптовалюта (2 упоминаний(-я))
16. Трансформация (2 упоминаний(-я))
17. Банк (2 упоминаний(-я))
18. Сфера (2 упоминаний(-я))
19. Диджитализация (2 упоминаний(-я))
20. Power (2 упоминаний(-я))
21. Ms (2 упоминаний(-я))
22. Excel (2 упоминаний(-я))
23. Bi (2 упоминаний(-я))
24. Надстройка (2 упоминаний(-я))
25. Pivot (2 упоминаний(-я))
26. Авс (2 упоминаний(-я))
27. Отч (2 упоминаний(-я))
28. Xyz (2 упоминаний(-я))
29. Сводный (2 упоминаний(-я))
30. Таблица (2 упоминаний(-я))


Направление — «Прикладн

In [362]:
for napr_kw_data in counter_keywords_all_naprs:
  vizual_popular_themes(napr_kw_data[1], f'Популярные темы/слова в РПД по направлению «{napr_kw_data[0]}»')

### Популярные темы по всем РПД

In [363]:

keywords_all_list = list(chain.from_iterable(rpd_kw_data.values()))
topic_counter = Counter(keywords_all_list)
counter_keywords_all = []

print("Наиболее популярные темы/слова:")
for (ind, (topic, count)) in enumerate(topic_counter.most_common(50)):
  counter_keywords_all.append((topic, count))
  print(f"{ind+1}. {topic.capitalize()} ({count} упоминаний(-я))")

print()
vizual_popular_themes(counter_keywords_all, 'Популярные темы/слова в РПД')

Наиболее популярные темы/слова:
1. Машинный (44 упоминаний(-я))
2. Python (44 упоминаний(-я))
3. Анализ (43 упоминаний(-я))
4. Технология (41 упоминаний(-я))
5. Сеть (40 упоминаний(-я))
6. Язык (38 упоминаний(-я))
7. Финансовый (36 упоминаний(-я))
8. Цифровой (32 упоминаний(-я))
9. Библиотека (29 упоминаний(-я))
10. Регрессия (29 упоминаний(-я))
11. Моделирование (28 упоминаний(-я))
12. Информационный (28 упоминаний(-я))
13. Нейронный (26 упоминаний(-я))
14. Архитектура (26 упоминаний(-я))
15. Модуль (26 упоминаний(-я))
16. Инструкция (25 упоминаний(-я))
17. Линейный (25 упоминаний(-я))
18. Граф (25 упоминаний(-я))
19. Классификация (24 упоминаний(-я))
20. Алгоритм (24 упоминаний(-я))
21. Сервис (22 упоминаний(-я))
22. Бизнес (22 упоминаний(-я))
23. Использование (21 упоминаний(-я))
24. Уравнение (21 упоминаний(-я))
25. Компьютерный (21 упоминаний(-я))
26. Принцип (21 упоминаний(-я))
27. Проектирование (21 упоминаний(-я))
28. Специфика (21 упоминаний(-я))
29. Формула (20 упоминаний(-я)

## Метрика относительной близости РПД

In [137]:
def rpd_similarity(filename_rpd1, filename_rpd2):
  keywords_rpd1, keywords_rpd2 = rpd_kw_data[filename_rpd1], rpd_kw_data[filename_rpd2]
  similar_keywords = [word_rpd1 for word_rpd1, word_rpd2 in zip(keywords_rpd1, keywords_rpd2) if word_rpd1 == word_rpd2]
  total_keywords = set(keywords_rpd1 + keywords_rpd2)

  return round((len(similar_keywords) / len(total_keywords)) * 100, 2)

### Генерация тестов подсчёта метрики

In [346]:
# генерируем всевозможные комбинации пар из 2 РПД из общего списка
# и проверяем количество
len(list(rpd_kw_data.keys())), len(list(itertools.combinations(list(rpd_kw_data.keys()), 2)))

(406, 82215)

In [349]:
for combo in itertools.combinations(list(rpd_kw_data.keys()), 2):
  if combo[0] != combo[1] and 70 < rpd_similarity(combo[0], combo[1]) < 100:
    print(combo)
    print(rpd_similarity(combo[0], combo[1]))

('Комп мод ЭиФС.docx', 'Комп мод ЭиФС, 2022.docx')
87.5
('Комп мод ЭиФС.docx', 'Компьютерное МЭиФС, ИБ, 2023.docx')
87.5
('Комп мод ЭиФС.docx', 'Компьютерное МЭиФС, ПИ, DO-И, 2023.docx')
87.5
('Алгоритмы и СД в яз Python, БИ.docx', 'Алг и СД в яз Python (БИ, 2022).docx')
80.0
('Алгоритмы и СД в яз Python, БИ.docx', 'А и СД в яз Python, БИ, 2023.docx')
80.0
('Алгоритмы и СД в яз Python, БИ.docx', 'А и СД в яз Python, Эк, 2023.docx')
80.0
('Веб-разработка, 2021.docx', 'Веб - разработка, ПИ, 2022.docx')
86.67
('Имитационное и АМ, 2021.docx', 'Имитационное и АМ, ПМИ, 2021.docx')
86.67
('Практик по прогр (ПИ, 2022).docx', 'Практик по программ, ПИ, 2023.docx')
87.5
('Практик по прогр (ПИ, 2022).docx', 'Практик по программ, МиКН, 2023.docx')
87.5
('Практик по прогр (ПИ, 2022).docx', 'Практик по программ, ПИнж, 2023.docx')
87.5
('Практик по прогр (ПМИ, 2022).docx', 'Практик по программ, ПИ, 2023.docx')
87.5
('Практик по прогр (ПМИ, 2022).docx', 'Практик по программ, МиКН, 2023.docx')
87.5
('Пр

### Фиксация результатов теста

In [350]:
rpd_couples = [
    (
        ('Современные ТГО, ИВТ.docx', 'Современные ТГО, Эк.docx'),
        rpd_similarity('Современные ТГО, ИВТ.docx', 'Современные ТГО, Эк.docx')
    ),
    (
        ('Финтех, ПИ.docx', 'Финтех, ПИ, 2022.docx'),
        rpd_similarity('Финтех, ПИ.docx', 'Финтех, ПИ, 2022.docx')
    ),
    (
        ('Комп мод ЭиФС.docx', 'Компьютерное МЭиФС, ПИ, DO-И, 2023.docx'),
        rpd_similarity('Комп мод ЭиФС.docx', 'Компьютерное МЭиФС, ПИ, DO-И, 2023.docx')
    ),
    (
        ('Имитационное и АМ, ПМИ, 2021.docx', 'Имитационное и АМ, 2021.docx'),
        rpd_similarity('Имитационное и АМ, ПМИ, 2021.docx', 'Имитационное и АМ, 2021.docx')
    ),
    (
        ('Практик по программ, ПИ, 2023.docx', 'Практик по прогр (ПМИ, 2022).docx'),
        rpd_similarity('Практик по программ, ПИ, 2023.docx', 'Практик по прогр (ПМИ, 2022).docx')
    ),
    (
        ('Интерактивные ТСКП, 2022.docx', 'Рекоменд СиКФ, ПМИ, 2023.docx'),
        rpd_similarity('Интерактивные ТСКП, 2022.docx', 'Рекоменд СиКФ, ПМИ, 2023.docx')
    ),
    (
        ('Рекоменд СиКФ, ПМИ, 2023.docx', 'Реком системы.docx'),
        rpd_similarity('Рекоменд СиКФ, ПМИ, 2023.docx', 'Реком системы.docx')
    ),
    (
        ('Визуализация данных, 2021, 2022.docx', 'Дискрет матем (ИБ, 2022).docx'),
        rpd_similarity('Визуализация данных, 2021, 2022.docx', 'Дискрет матем (ИБ, 2022).docx')
    ),
    (
        ('Основы МЗ, ПМИ, 2023.docx', 'Современные ТКЗ, ИВТ.docx'),
        rpd_similarity('Основы МЗ, ПМИ, 2023.docx', 'Современные ТКЗ, ИВТ.docx')
    ),
    (
        ('Прикладные ЗМО, ИБ, 2023.docx', 'Введение в НоД, Ин, 2023.docx'),
        rpd_similarity('Прикладные ЗМО, ИБ, 2023.docx', 'Введение в НоД, Ин, 2023.docx')
    ),
    (
        ('Алг и СД в яз Python (БИ, 2022).docx', 'А и СД в яз Python, ПИ, 2023.docx'),
        rpd_similarity('Алг и СД в яз Python (БИ, 2022).docx', 'А и СД в яз Python, ПИ, 2023.docx')
    ),
    (
        ('Практик по программ, ПИ, 2023.docx', 'Алг и СД в яз Python (ПИ, 2022).docx'),
        rpd_similarity('Практик по программ, ПИ, 2023.docx', 'Алг и СД в яз Python (ПИ, 2022).docx')
    ),
    (
        ('Алг и СД в яз Python (ПИ, 2022).docx', 'Алгоритмы и СД в яз Python, БИ.docx'),
        rpd_similarity('Алг и СД в яз Python (ПИ, 2022).docx', 'Алгоритмы и СД в яз Python, БИ.docx')
    ),
    (
        ('Финтех (Эк).docx', 'Финтех, ПМИ, 2023.docx'),
        rpd_similarity('Финтех (Эк).docx', 'Финтех, ПМИ, 2023.docx')
    ),
    (
        ('Алгоритмы и СД в яз Python, ПМИ.docx', 'А и СД в яз Python, Эк, 2023.docx'),
        rpd_similarity('Алгоритмы и СД в яз Python, ПМИ.docx', 'А и СД в яз Python, Эк, 2023.docx')
    ),
    (
        ('Финтех, ИБ,2022.docx', 'Финтех (ФРО).docx'),
        rpd_similarity('Финтех, ИБ,2022.docx', 'Финтех (ФРО).docx')
    ),
    (
        ('Финтех, ИБ, 2021.docx', 'Финтех (ФРО).docx'),
        rpd_similarity('Финтех, ИБ, 2021.docx', 'Финтех (ФРО).docx')
    ),
    (
        ('Дискрет матем, ПМИ.docx', 'Дискрет матем (ПИ, 2022).docx'),
        rpd_similarity('Дискрет матем, ПМИ.docx', 'Дискрет матем (ПИ, 2022).docx')
    ),
    (
        ('Пред аналитика БД, м, ПИ, 2022.docx', 'Пред аналитика БД, ПИ, 2022.docx'),
        rpd_similarity('Пред аналитика БД, м, ПИ, 2022.docx', 'Пред аналитика БД, ПИ, 2022.docx')
    ),
    (
        ('Основы ВР, ПИ, 2023.docx', 'Web-программ, ПИ, 2023.docx'),
        rpd_similarity('Основы ВР, ПИ, 2023.docx', 'Web-программ, ПИ, 2023.docx')
    ),
    (
        ('Основы ПИС, ПМИ, 2023.docx', 'Проектирование ИС, ПИ, 2023.docx'),
        rpd_similarity('Основы ПИС, ПМИ, 2023.docx', 'Проектирование ИС, ПИ, 2023.docx')
    ),
    (
        ('Прогр на яз Python и SQL, Эк.docx', 'Програм. на языках PYTHON и SQL,Эк, КФУ.docx'),
        rpd_similarity('Прогр на яз Python и SQL, Эк.docx', 'Програм. на языках PYTHON и SQL,Эк, КФУ.docx')
    ),
    (
        ('Алгоритмы и СД в яз Python, БИ.docx', 'А и СД в яз Python, БИ, 2023.docx'),
        rpd_similarity('Алгоритмы и СД в яз Python, БИ.docx', 'А и СД в яз Python, БИ, 2023.docx')
    ),
    (
        ('Финтех, Эк, 2022.docx', 'Финтех, Мен,2022.docx'),
        rpd_similarity('Финтех, Эк, 2022.docx', 'Финтех, Мен,2022.docx')
    ),
    (
        ('ТВиМС, ИБ, 2022.docx', 'ТВиМС, ПИ, 2022.docx'),
        rpd_similarity('ТВиМС, ИБ, 2022.docx', 'ТВиМС, ПИ, 2022.docx')
    ),
    (
        ('А и СД в яз Python, ИБ, 2023.docx', 'Алг и СД в яз Python (ИБ, 2022).docx'),
        rpd_similarity('А и СД в яз Python, ИБ, 2023.docx', 'Алг и СД в яз Python (ИБ, 2022).docx')
    ),
    (
        ('Фин универ, ИБ, 2023.docx', 'ФУ_ИиС (ПМИ, 2022).docx'),
        rpd_similarity('Фин универ, ИБ, 2023.docx', 'ФУ_ИиС (ПМИ, 2022).docx')
    ),
    (
        ('Инстр Power BI и Excel, 2021.docx', 'Инстр Power BI и Excel в лог, 2022.docx'),
        rpd_similarity('Инстр Power BI и Excel, 2021.docx', 'Инстр Power BI и Excel в лог, 2022.docx')
    ),
    (
        ('Современные ТКЗ, Эк.docx', 'Современные ТКЗ, Эк.docx'),
        rpd_similarity('Современные ТКЗ, Эк.docx', 'Современные ТКЗ, Эк.docx')
    )
]

for couple in sorted(rpd_couples, key=lambda elem: -elem[1]):
  print(f'rpd_similarity{couple[0]} = {couple[1]}\n')

rpd_similarity('Финтех, ИБ, 2021.docx', 'Финтех (ФРО).docx') = 100.0

rpd_similarity('Финтех, Эк, 2022.docx', 'Финтех, Мен,2022.docx') = 100.0

rpd_similarity('ТВиМС, ИБ, 2022.docx', 'ТВиМС, ПИ, 2022.docx') = 100.0

rpd_similarity('А и СД в яз Python, ИБ, 2023.docx', 'Алг и СД в яз Python (ИБ, 2022).docx') = 100.0

rpd_similarity('Фин универ, ИБ, 2023.docx', 'ФУ_ИиС (ПМИ, 2022).docx') = 100.0

rpd_similarity('Инстр Power BI и Excel, 2021.docx', 'Инстр Power BI и Excel в лог, 2022.docx') = 100.0

rpd_similarity('Современные ТКЗ, Эк.docx', 'Современные ТКЗ, Эк.docx') = 100.0

rpd_similarity('Комп мод ЭиФС.docx', 'Компьютерное МЭиФС, ПИ, DO-И, 2023.docx') = 87.5

rpd_similarity('Практик по программ, ПИ, 2023.docx', 'Практик по прогр (ПМИ, 2022).docx') = 87.5

rpd_similarity('Имитационное и АМ, ПМИ, 2021.docx', 'Имитационное и АМ, 2021.docx') = 86.67

rpd_similarity('Алгоритмы и СД в яз Python, БИ.docx', 'А и СД в яз Python, БИ, 2023.docx') = 80.0

rpd_similarity('Финтех, ПИ.docx', 'Финтех